In [2]:
import numpy as np
import pandas as pd

In [154]:
data = pd.read_csv('data/initial.csv.gz', compression='gzip', low_memory=True)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (0,18,48,58,117,128,129,130,133,134,135,138,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [155]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1646801 entries, 0 to 1646800
Columns: 150 entries, id to settlement_term
dtypes: float64(113), object(37)
memory usage: 1.8+ GB


In [156]:
data = data.loc[data['loan_status'].isin(['Fully Paid', 'Charged Off'])]
data['loan_status'] = data['loan_status'].apply(lambda v: np.float(v == 'Charged Off'))
data['term'] = data['term'].apply(lambda s:np.float(s[1:3]))

In [157]:
cols = [
    'annual_inc', 
    'delinq_2yrs',
    'loan_status',
    'open_acc',
    'revol_bal',
    'total_acc',
    'term',
    'loan_amnt',
    'installment',
#     'collection_recovery_fee',
    'int_rate',
    'tot_cur_bal',
    'max_bal_bc'
]

In [158]:
data = data[cols]
data.shape

(814986, 12)

In [160]:
corr = data.corr()['loan_status']
corr

annual_inc    -0.046685
delinq_2yrs    0.021610
loan_status    1.000000
open_acc       0.034652
revol_bal     -0.013160
total_acc     -0.011187
term           0.177708
loan_amnt      0.064139
installment    0.046291
int_rate       0.247815
tot_cur_bal   -0.076593
max_bal_bc    -0.025957
Name: loan_status, dtype: float64

In [161]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [162]:
x = data.drop(labels=['loan_status'], axis=1)
y = data['loan_status']

In [163]:
from sklearn.model_selection import train_test_split


In [164]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [165]:
from sklearn.preprocessing import Imputer
imputer = Imputer().fit(x_train)
x_train = pd.DataFrame(imputer.transform(x_train), columns=x_train.columns)
x_test  = pd.DataFrame(imputer.transform(x_test),  columns=x_test.columns)

In [147]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
x_test  = pd.DataFrame(scaler.transform(x_test),  columns=x_test.columns)

In [166]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef, make_scorer

In [167]:
x_train.shape

(733487, 11)

In [150]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


In [168]:
model = SGDClassifier()
model.fit(x_train, y_train)

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [169]:
y_pred = model.predict(x_test)

In [170]:
print('Confusion matrix')
print(confusion_matrix(y_test, y_pred),'\n')
print('Classification report')
print(classification_report(y_test, y_pred, digits=3))
print('MCC = ',matthews_corrcoef(y_test, y_pred))
print('Accuracy = ',accuracy_score(y_test, y_pred))

Confusion matrix
[[64806    85]
 [16556    52]] 

Classification report
             precision    recall  f1-score   support

        0.0      0.797     0.999     0.886     64891
        1.0      0.380     0.003     0.006     16608

avg / total      0.712     0.796     0.707     81499

MCC =  0.01790693710609501
Accuracy =  0.7958134455637492


In [171]:
from sklearn.ensemble import GradientBoostingClassifier

In [172]:
model = GradientBoostingClassifier()

In [173]:
model.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [174]:
y_pred = model.predict(x_test)

In [175]:
print('Confusion matrix')
print(confusion_matrix(y_test, y_pred),'\n')
print('Classification report')
print(classification_report(y_test, y_pred, digits=3))
print('MCC = ',matthews_corrcoef(y_test, y_pred))
print('Accuracy = ',accuracy_score(y_test, y_pred))

Confusion matrix
[[64049   842]
 [15583  1025]] 

Classification report
             precision    recall  f1-score   support

        0.0      0.804     0.987     0.886     64891
        1.0      0.549     0.062     0.111     16608

avg / total      0.752     0.798     0.728     81499

MCC =  0.1312305846625252
Accuracy =  0.7984637848317158


In [27]:
from sklearn.ensemble import RandomForestClassifier


In [28]:
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [29]:
print('Confusion matrix')
print(confusion_matrix(y_test, y_pred),'\n')
print('Classification report')
print(classification_report(y_test, y_pred, digits=3))
print('MCC = ',matthews_corrcoef(y_test, y_pred))
print('Accuracy = ',accuracy_score(y_test, y_pred))

Confusion matrix
[[64366   410]
 [ 6966  9757]] 

Classification report
             precision    recall  f1-score   support

        0.0      0.902     0.994     0.946     64776
        1.0      0.960     0.583     0.726     16723

avg / total      0.914     0.909     0.901     81499

MCC =  0.7053269816745702
Accuracy =  0.9094958220346262
